In [12]:
!pip install gymnasium[atari]

In [13]:
!pip install keras
!pip install tensorflow
!pip install --user Pillow
!pip install --user h5py
!pip install --user keras-rl
!pip install gymnasium[atari]
!pip install keras-rl2

In [14]:
import numpy as np
!pip install gymnasium
!pip install gym[atari,accept-rom-license]
import gymnasium as gym
#import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [18]:
# Initialize Gym environment
ENV_NAME = 'Breakout-v0'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

C:\Users\clayb\AppData\Roaming\Python\Python310\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


(693650678, 2973253328)

In [19]:
# Number of actions
nb_actions = env.action_space.n

In [20]:
# Define the model architecture
input_shape = (84, 84)
window_length = 4

model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [36]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000, window_length=3)
policy = EpsGreedyQPolicy(eps=0.1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

In [34]:
# Train the agent
dqn.fit(env, nb_steps=1750000, visualize=False, verbose=2)

NameError: name 'dqn' is not defined

In [35]:
# Save the final policy network
dqn.save_weights('policy.h5', overwrite=True)

NameError: name 'dqn' is not defined

# Play Environment

In [ ]:
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = GreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Load the trained weights
dqn.load_weights('policy.h5')

In [ ]:
# Evaluate the agent
dqn.test(env, nb_episodes=10, visualize=True)